In [1]:
/// \section*{What's new}
///
/// \begin{itemize}
/// \item Creating numerical grids
/// \item Projecting functions onto the DG space and further evaluation
/// \item Performing a hp-convergence study
/// \end{itemize}
///
/// \section*{Prerequisites}
/// No \BoSSS{} specific prerequisites are needed to complete this tutorial.
///
/// \section{Problem statement}
/// First, we define two functions: $g_1$ is continuous, $g_2$ has a 
/// discontinuity at $\vec{X} = \pi$ in the first derivative
/// \begin{align*}
/// g_1 =& \sin(\vec{X}), \\
/// g_2 =& \vert(\sin(\vec{X})\vert\, .
/// \end{align*}
/// The function argument is a vector $\vec{X}$, consisting only of one entry 
/// since we are working in a one dimensional space. 
/// \BoSSS{} however supports 1D, 2D and 3D, so the spatial coordinate 
/// is a general vector.
///
/// \section{Solution within the BoSSS framework}
///
/// To initialize, we have to call \code{restart}:\textsl{}

In [1]:
#r "C:\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
using NUnit.Framework;

In [1]:
/// \subsection{Plotting the functions} First, we plot the functions that are defined above over the interval $(0 ,2 \pi)$ with 
/// 1000 sampling points using a Gnuplot-object.
/// %=======================================================================================

In [1]:
Func<double[],double> g1 = (X => Math.Sin(X[0]));          
// continuous, smooth         
Func<double[],double> g2 = (X => Math.Abs(Math.Sin(X[0])));          
// continuous, non-smooth

In [1]:
/// We define equidistant sampling points...
double[] x = GenericBlas.Linspace(0, 2.0*Math.PI, 1000);

In [1]:
/// ...and compute the function values. In the loop, we have to convert the scalar \code{x[i]} into an array
/// with one element, since \code{g1} has to be feed with arrays.
double[] g1_values = new double[x.Length];
for(int i = 0; i < x.Length; i++) {
    g1_values[i] = g1(new[] { x[i]} );
}

In [1]:
/// Instead of loops, we can also use Linq-functions:
double[] g2_values = x.Select(x => g2(new []{ x })).ToArray();

In [1]:
/// For now, we are using the simple plotting interface, which supports
/// Matlab-Style format specifiers and color names. (More advanced plots
/// can be produced with \code{Plot2Ddata} and/or \code{Gnuplot} classes)
Plot(X1:x, Y1:g1_values, Name1:"function g1", Format1:"--red",
     X2:x, Y2:g2_values, Name2:"function g2", Format2:"-.blue");

In [1]:
/// Next, we create a grid which has a cell boundary exactly at the position of
/// the discontinuity of \code{g2}.
/// %=========================================================================================

In [1]:
var Nodes1 = new double[] {0, 2, Math.PI, 4.5, 2*Math.PI };         
var Grid1 = Grid1D.LineGrid(Nodes1);

In [1]:
/// We can get the total number of cells by using the following command:

In [1]:
Grid1.NumberOfCells

In [1]:
/// The recently created grid-object is not directly usable because it contains only the nodes of the grid. 
/// We have to create a \code{GridData}-object which provides all necessary transformation metrics, etc. .
/// %=======================================================================================

In [1]:
var gdata1 = new GridData(Grid1);

In [1]:
/// \subsection{Projection onto the DG space}
/// At this point, we are able to create the so-called \emph{DG fields} to approximate \code{g1}
/// on \code{grid1}. Therefore, we project \code{g1} onto \code{grid1} using polynomial orders
/// of $p=2$ and $p=8$.
/// % =======================================================================================

In [1]:
var g1_grid1_p2 = new SinglePhaseField(new Basis(gdata1, 2), "g1 with p2 at Grid 1");      
g1_grid1_p2.ProjectField(g1);

In [1]:
var g1_grid1_p8 = new SinglePhaseField(new Basis(gdata1, 8), "g1 with p8 at Grid 1");         
g1_grid1_p8.ProjectField(g1);

In [1]:
/// Now, let us plot the projected solution for $p=2$. 
/// By using the upsampling parameter, we can determine 
/// the amount of sampling points per cell.
/// %=======================================================================================

In [1]:
var upsampling = 20;

In [1]:
var gp1 = new Gnuplot();

In [1]:
gp1.PlotField(g1_grid1_p2,          
    new PlotFormat(lineColor: (LineColors)(1)),
    upsampling);

In [1]:
gp1.PlotNow(); // shows the plot

In [1]:
/// \subsection{Computing the $L^2$-error}
/// Next, we learn how to compute the $L^2$-error for both 
/// approximations of \code{g1} with different polynomial degrees:
/// %=======================================================================================

In [1]:
g1_grid1_p2.L2Error(g1);

In [1]:
g1_grid1_p8.L2Error(g1);

In [1]:
/// \subsection{Plotting the point-wise error}
/// Now, we plot the point-wise error for the approximation of \code{g1}
/// on \code{grid1} with a polynomial degree of 8.
/// %=======================================================================================

In [1]:
int K = 20; // number of points per cell         
var gp2 = new Gnuplot();         
gp2.PlotLogError(g1_grid1_p8, g1, "g1 with p8 at Grid 1", 20,          
    new PlotFormat(lineColor: (LineColors)(1)));         
gp2.PlotNow();

In [1]:
/// \subsection{Decay behavior of the DG modes for smooth and non-smooth functions}
/// We investigate the decay behavior of the DG modes for smooth and non-smooth 
/// functions. For this purpose, we create a second grid which has the
/// discontinuity of \code{g2} within a cell and project \code{g2} onto this grid
/// like mentioned above.
/// %=======================================================================================

In [1]:
var Nodes2      = new double[] {0, 2, 4.5, 2*Math.PI };         
var Grid2       = Grid1D.LineGrid(Nodes2);         
var gdata2      = new GridData(Grid2);         
var g2_grid2_p8 = new SinglePhaseField(new Basis(gdata2, 8), "g2_p8 at Grid2");         
g2_grid2_p8.ProjectField(g2);

In [1]:
/// The cell coordinates can be extracted by using the \code{Coordinates} parameter.

In [1]:
double[] cell1 = g2_grid2_p8.Coordinates.GetRow(1);          
    // coord. in cell 1 (with kink)

In [1]:
double[] cell0 = g2_grid2_p8.Coordinates.GetRow(0);          
    // coord. in cell 0 (smooth)

In [1]:
double[] cell2 = g2_grid2_p8.Coordinates.GetRow(2);          
    // coord. in cell 2 (smooth)

In [1]:
/// Only the absolute value shall be plotted. We use a for-loop to replace the data in 
/// \code{cell0}, \code{cell1} and \code{cell2} by their absolute values.
/// %=======================================================================================

In [1]:
for(int i = 0; i < cell0.Length; i++) {         
    cell0[i] = Math.Abs(cell0[i]);         
    cell1[i] = Math.Abs(cell1[i]);         
    cell2[i] = Math.Abs(cell2[i]);         
 }

In [1]:
Plot(X1:null, Y1:cell1, Name1:"disc. cell", Format1:"*-magenta",
     X2:null, Y2:cell0, Name2:"cell0",      Format2:"o-blue",
     X3:null, Y3:cell2, Name3:"cell2",      Format3:"o-red");

In [1]:
/// Note: Using a shortcut for the for-loop above, the absolute values in cell 0
/// can also be stored using the following command:

In [1]:
double[] cell0 = g2_grid2_p8.Coordinates.GetRow(0)         
    .Select(d => Math.Abs(d)).ToArray();

In [1]:
/// Now, we would like to plot the logarithm (use \code{Math.Log10(...)}) of the absolute
/// values of the DG coordinates.

In [1]:
Plot(X1:null, Y1:cell1, Name1:"disc. cell", Format1:"*-magenta",
     X2:null, Y2:cell0, Name2:"cell0",      Format2:"o-blue",
     X3:null, Y3:cell2, Name3:"cell2",      Format3:"o-red",
     logY:true);

In [1]:
///\subsection{Convergence study}

In [1]:
/// In this section, we learn how to perform a convergence study for \code{g2} 
/// for two different sequences of grid resolutions and different polynomial
/// orders. Therefore, we define two different sequences of grid resolutions:
/// %=======================================================================================

In [1]:
int[][] ResSeq = new int[2][];

In [1]:
/// Grid resolutions so that the kink in \code{g2} is located at a cell boundary:
ResSeq[0] = new int[] { 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048 };

In [1]:
/// Grid resolutions so that the kink in \code{g2} is located within a cell:
ResSeq[1] = new int[] { 3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047 };

In [1]:
/// We save our errors into a multidimensional array by looping over 
/// \begin{enumerate}
/// \item the resolution sequence
/// \item the polynomial order
/// \item the resolution
/// \end{enumerate}
var Errors = MultidimensionalArray.Create(2, 5, ResSeq[0].Length);

In [1]:
for(int i = 0; i < 2; i++) { // loop over the resolution sequence         
    for(int p = 0; p <= 4; p++) { // loop over polynomial orders         
        for(int k = 0; k < ResSeq[i].Length; k++) { // loop over different resolutions        
 
            Console.Write("polynomial order {1}"+         
            ",\tResolution {0}... ", ResSeq[i][k], p);         
 
            var grid  = Grid1D.LineGrid(GenericBlas.Linspace(0,         
            2.0*Math.PI, ResSeq[i][k] + 1));           
                 // number of nodes == number of cells + 1         
 
            var gData = new GridData(grid);         
 
            var g2_h  = new SinglePhaseField(new Basis(gData, p));         
 
            g2_h.ProjectField(g2);         
 
            Errors[i,p,k] = g2_h.L2Error(g2);         
 
            Console.WriteLine("\tdone: L2 error is {0:0.###e-00}.", Errors[i,p,k]);         
        }         
    }         
 }

In [1]:
/// NUnit test (few random tests) BoSSScmdSilent
Assert.LessOrEqual(Errors[1,4,9],8E-06);       
Assert.LessOrEqual(Errors[1,3,9],7.5E-06);       
Assert.LessOrEqual(Errors[1,2,9],2E-05);       
Assert.LessOrEqual(Errors[1,1,9],2E-05);       
Assert.LessOrEqual(Errors[0,3,9],1E-12);       
Assert.LessOrEqual(Errors[0,3,9],1E-12);       
Assert.LessOrEqual(Errors[1,4,0],0.25);       
Assert.LessOrEqual(Errors[0,0,0],0.2);       
Assert.LessOrEqual(Errors[0,3,0],1E-03);

In [1]:
/// We plot the error for the grids which have the kink at the cell boundary,
/// there we reach spectral convergence:

In [1]:
var hValues = ResSeq[0].Select(J => Math.PI*2.0/J);
Plot(X1:hValues, Y1:Errors.ExtractSubArrayShallow(0,0,-1).To1DArray(),
     Name1:"grid1,p0", Format1:"o-red",
     X2:hValues, Y2:Errors.ExtractSubArrayShallow(0,1,-1).To1DArray(),
     Name2:"grid1,p1", Format2:"o-blue",
     X3:hValues, Y3:Errors.ExtractSubArrayShallow(0,2,-1).To1DArray(),
     Name3:"grid1,p2", Format3:"o-green",
     X4:hValues, Y4:Errors.ExtractSubArrayShallow(0,3,-1).To1DArray(),
     Name4:"grid1,p3", Format4:"o-magenta",
     X5:hValues, Y5:Errors.ExtractSubArrayShallow(0,4,-1).To1DArray(),
     Name5:"grid1,p4", Format5:"o-orange",
     logX:true, logY:true);

In [1]:
/// Now we plot the error for the grids which have the kink within a cell;
/// due to the low regularity, the convergence of the DG method
/// degenerates:

In [1]:
var hValues = ResSeq[0].Select(J => Math.PI*2.0/J);
Plot(X1:hValues, Y1:Errors.ExtractSubArrayShallow(1,0,-1).To1DArray(),
     Name1:"grid1,p0", Format1:"o-red",
     X2:hValues, Y2:Errors.ExtractSubArrayShallow(1,1,-1).To1DArray(),
     Name2:"grid1,p1", Format2:"o-blue",
     X3:hValues, Y3:Errors.ExtractSubArrayShallow(1,2,-1).To1DArray(),
     Name3:"grid1,p2", Format3:"o-green",
     X4:hValues, Y4:Errors.ExtractSubArrayShallow(1,3,-1).To1DArray(),
     Name4:"grid1,p3", Format4:"o-magenta",
     X5:hValues, Y5:Errors.ExtractSubArrayShallow(1,4,-1).To1DArray(),
     Name5:"grid1,p4", Format5:"o-orange",
     logX:true, logY:true);

In [1]:
/// \section{Advanced topics} This tutorial addressed the very basics of setting up a \BoSSS{}~application, namely grid instantiation, the $L^2$-projection of functions onto the DG space and performing a spatial convergence study. Where do you go from here? We recommend that you continue with other relevant basics as provided in the tutorials dealing with the creation of a spatial operator, explicit time integration and the implementation of numerical fluxes.